In [1]:
import os
import sys
from pathlib import Path

import networkx as nx
import cvxpy as cp
import numpy as np

FILE = Path().resolve()
ROOT = FILE.parents[0]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from utils.utils_3d import *


In [2]:
graph_name = "graph_02"

# Define the regularization parameter
alpha = 0.5


In [3]:
path = f"{ROOT}/graphs/3d/{graph_name}.txt"
g1 = load_graph(path)

# Get the positions of each node
pos = nx.get_node_attributes(g1, "pos")
num_nodes = g1.number_of_nodes()

# Compute moving nodes
moving_nodes = {}
for node in g1.nodes():
    neighbors = list(g1.neighbors(node))
    if len(neighbors) == 2:
        moving_nodes[node] = neighbors

# Create the optimization variables
opt_path = cp.Variable((num_nodes, 3))

# Define the objective function to minimize the curvature and distance
curvature = 0
for node, (prev_node, next_node) in moving_nodes.items():
    curvature += cp.sum_squares(
        opt_path[node, :] - 0.5 * (opt_path[prev_node, :] + opt_path[next_node, :])
    )

obj = curvature + alpha * cp.sum_squares(opt_path - np.array(list(pos.values())))

# Define the constraints for fixed nodes
constraints = []
for node, position in pos.items():
    if len(list(g1.neighbors(node))) != 2:
        constraints.append(opt_path[node, :] == np.array(position))

# Create the optimization problem
problem = cp.Problem(cp.Minimize(obj), constraints)

# Solve the problem
problem.solve()

# Retrieve the optimized path
optimized_pos = {node: tuple(opt_path[node, :].value) for node in g1.nodes()}

# Create optimized graph
g2 = create_graph(optimized_pos, g1.edges())

# Draw the original and optimized graphs together
save_name = f"{ROOT}/results/3d/{graph_name}.html"
plot_graph(g1, g2, alpha, save_name)
